In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [10]:
final_df= pd.read_csv("../Data/final_merged_df.csv", delimiter=',')
final_df.head(5)

,Wikipedia movie ID,char A,action,char B,combined_action,lemmatized_action,char A type,char B type,freebase_movie_id,movie_name,...,anger,love,fear,care,grief,mentor,confrontation,heartbreak,collaborate,emotional
0,31186339,Peeta Mellark,"['reveal', 'love', 'for']",Katniss Everdeen,reveal love for,"['reveal', 'love', 'for']",side,side,/m/0gkz15s,The Hunger Games,...,0,1,0,0,0,0,0,0,0,0
1,31186339,Katniss Everdeen,['discover'],Peeta Mellark,discover,['discover'],side,side,/m/0gkz15s,The Hunger Games,...,0,0,0,0,0,0,0,0,0,0
2,31186339,Rue,['call'],Katniss Everdeen,call,['call'],side,side,/m/0gkz15s,The Hunger Games,...,0,0,0,0,0,0,0,0,0,0
3,31186339,Katniss Everdeen,['find'],Rue,find,['find'],side,side,/m/0gkz15s,The Hunger Games,...,0,0,0,0,0,0,0,0,0,0
4,31186339,Katniss Everdeen,"['comfort', 'the', 'dying']",Rue,comfort the dying,"['comfort', 'the', 'die']",side,side,/m/0gkz15s,The Hunger Games,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# group by wiki_id and count the number of times character appears in a movie
# Initialize the dictionary to store results
movie_results = {}

# Iterate through unique movie IDs
for movie_id in final_df['Wikipedia movie ID'].unique():
    # Filter the DataFrame for the current movie
    movie_df = final_df[final_df['Wikipedia movie ID'] == movie_id]
    
    # Concatenate char A and char B columns and count occurrences
    char_counts = pd.concat([movie_df['char A'], movie_df['char B']]).value_counts().sort_values(ascending=False)
    
    # Get the most frequently mentioned character
    main_char = char_counts.idxmax()
    second_char = char_counts.index[1] if len(char_counts) > 1 else None
    
    # Store the result in the dictionary
    movie_results[movie_id] = {main_char: 'main'}
    if second_char:
        movie_results[movie_id][second_char] = 'main'

# Print the results
print(movie_results)

{31186339: {'Katniss Everdeen': 'main', 'Peeta Mellark': 'main'}, 20663735: {'Marancheri Induchoodan': 'main', 'Manapally': 'main'}, 2231378: {'Kid': 'main', 'Charley': 'main'}, 595909: {'Lindy Chamberlain': 'main', 'Azaria': 'main'}, 5272176: {'Thomas': 'main', 'Baldwin': 'main'}, 1952976: {'Cecilia': 'main', 'Natasha': 'main'}, 24225279: {'Hannah': 'main', 'Dominic': 'main'}, 2462689: {"Long John Willoughby - 'John Doe'": 'main', 'Ann Mitchell': 'main'}, 20532852: {'Buzz': 'main', 'Woody': 'main'}, 15401493: {'Bo': 'main', 'Simona': 'main'}, 1335380: {'Ari Ben Canaan': 'main', 'Dov Landau': 'main'}, 1480747: {'Kid': 'main', 'Miles': 'main'}, 4018288: {"Debbie's Mom": 'main'}, 4596602: {'Arabella Simone': 'main', 'Gabriel Darbeaux': 'main'}, 9252321: {'Zachariah': 'main', 'Marumu': 'main'}, 8388648: {'Madhav': 'main', 'Amrutha': 'main'}, 10644072: {'Bhagwaan Dada': 'main', 'Govinda': 'main'}, 33269085: {'Roger': 'main', 'Kamla Patel': 'main'}, 2254491: {'Sita': 'main', 'Anita Verma': 

In [15]:
for idx, row in final_df.iterrows():
    # Get the main character of the movie
    main_char = movie_results[row['Wikipedia movie ID']]
    
    # Check if the current character is the main character
    if row['char A'] in main_char:
        final_df.loc[idx, 'char A type'] = 'main'
    if row['char B'] in main_char:
        final_df.loc[idx, 'char B type'] = 'main'

# Print the updated DataFrame
final_df.head()

,Wikipedia movie ID,char A,action,char B,combined_action,lemmatized_action,char A type,char B type,freebase_movie_id,movie_name,...,anger,love,fear,care,grief,mentor,confrontation,heartbreak,collaborate,emotional
0,31186339,Peeta Mellark,"['reveal', 'love', 'for']",Katniss Everdeen,reveal love for,"['reveal', 'love', 'for']",main,main,/m/0gkz15s,The Hunger Games,...,0,1,0,0,0,0,0,0,0,0
1,31186339,Katniss Everdeen,['discover'],Peeta Mellark,discover,['discover'],main,main,/m/0gkz15s,The Hunger Games,...,0,0,0,0,0,0,0,0,0,0
2,31186339,Rue,['call'],Katniss Everdeen,call,['call'],side,main,/m/0gkz15s,The Hunger Games,...,0,0,0,0,0,0,0,0,0,0
3,31186339,Katniss Everdeen,['find'],Rue,find,['find'],main,side,/m/0gkz15s,The Hunger Games,...,0,0,0,0,0,0,0,0,0,0
4,31186339,Katniss Everdeen,"['comfort', 'the', 'dying']",Rue,comfort the dying,"['comfort', 'the', 'die']",main,side,/m/0gkz15s,The Hunger Games,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# save the updated dataframe
final_df.to_csv("../Data/final_merged_df.csv", index=False)

### Graph clustering

In [1]:
import plotly.graph_objects as px
import numpy as np
 
 
np.random.seed(52)
 
random_x = np.random.randint(1, 120, 100)
random_y = np.random.randint(1, 120, 100)
 
fig = px.Figure(data=[px.Scatter(
	x=random_x,
	y=random_y,
	mode='markers',)
])
 
fig.update_layout(
	updatemenus=[
		dict(
		buttons=list([
			dict(
			args=["type", "scatter"],
			label="Scatter Plot",
			method="restyle"
			),
                        dict(
                        args=["type", "box"],
                        label="Box Plot",
                        method="restyle"
                        ),
			dict(
			args=["type", "bar"],
			label="Bar Plot",
			method="restyle"
			)
			]),
			direction="down",
		    ),
	      ],
      title = "Dropdown Menu in plotly Graph Using Python"
)
 
fig.show()